In [5]:
# Model Selection
# ===============
# This notebook compares the scores of different model algorithms, 
# preprocessing methods, and classification methdos. 
scores_list <- list(
    list('ridge regression', 'robust scaling', TRUE, 0.7742, 0.8478, 0.7358, 0.825, 0.5608),
    list('logistic regression', 'robust scaling', TRUE, 0.7742, 0.8333, 0.7547, 0.8, 0.5547),
    list('linear discriminant analysis', 'robust scaling', TRUE, 0.7742, 0.8333, 0.7547, 0.8, 0.5547),
    list('quadratic discriminant analysis', 'normalize', TRUE, 0.7849, 0.8367, 0.7736, 0.8, 0.5736),
    list('quadratic discriminant analysis', 'standard scaling', TRUE, 0.8065, 0.8302, 0.8302, 0.775, 0.6052),
    list('quadratic discriminant analysis', 'robust scaling', TRUE, 0.8065, 0.8302, 0.8302, 0.775, 0.6052),
    list('quadratic discriminant analysis', 'pca', TRUE, 0.8387, 0.8519, 0.8679, 0.8, 0.6679),
    list('support vector machine', 'standard scaling', TRUE, 0.7957, 0.84, 0.7925, 0.8, 0.5925),
    list('support vector machine', 'robust scaling', TRUE, 0.7849, 0.8367, 0.7736, 0.8, 0.5736),
    list('support vector machine', 'pca', TRUE, 0.8065, 0.8431, 0.8113, 0.8, 0.6113),
    list('random forest', 'standard scaling', TRUE, 0.7742, 0.82, 0.7736, 0.775, 0.5486),
    list('random forest', 'robust scaling', TRUE, 0.7849, 0.8235, 0.7925, 0.775, 0.5675),
    list('random forest', 'pca', TRUE, 0.828, 0.8776, 0.8113, 0.85, 0.6613))
 

scores <- data.frame(do.call(rbind.data.frame, scores_list))
colnames(scores) <- c('model', 'preprocessing', 'includes fbs', 'accuracy', 'precision', 'sensitivity', 'specificity', 'informedness')
scores

,model,preprocessing,includes fbs,accuracy,precision,sensitivity,specificity,informedness
,<chr>,<chr>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,ridge regression,robust scaling,TRUE,0.7742,0.8478,0.7358,0.825,0.5608
21,logistic regression,robust scaling,TRUE,0.7742,0.8333,0.7547,0.800,0.5547
3,linear discriminant analysis,robust scaling,TRUE,0.7742,0.8333,0.7547,0.800,0.5547
4,quadratic discriminant analysis,normalize,TRUE,0.7849,0.8367,0.7736,0.800,0.5736
5,quadratic discriminant analysis,standard scaling,TRUE,0.8065,0.8302,0.8302,0.775,0.6052
6,quadratic discriminant analysis,robust scaling,TRUE,0.8065,0.8302,0.8302,0.775,0.6052
7,quadratic discriminant analysis,pca,TRUE,0.8387,0.8519,0.8679,0.800,0.6679
8,support vector machine,standard scaling,TRUE,0.7957,0.8400,0.7925,0.800,0.5925
9,support vector machine,robust scaling,TRUE,0.7849,0.8367,0.7736,0.800,0.5736
